In [15]:
%cd /Users/ben/prj/fab/_scripts

/Users/ben/prj/fab/_scripts


In [14]:
## FROM SYNC_CANS.PY
#!/usr/bin/env python

import fill_untappd
import yaml, json, sqlite3,  dateparser,os, sys
from pprint import pprint
from airtable import Airtable
import pandas as pd


def main():
    fill_untappd.run()
    conn = sqlite3.connect("untappd.db")
    df = pd.read_sql_query("select * from checkins;", conn)
    jsons  = df.json.apply(lambda j: json.loads(j))

    venues = jsons.apply(
        lambda x: 
        pd.concat([
            pd.Series(x["venue"]),
            pd.Series(x["user"])[["user_name"]],
            pd.Series(x["beer"]),
            pd.Series(x)[["checkin_id","created_at","rating_score"]]
            ])
            )

    venues_info = pd.concat([venues, 
    pd.concat([
        venues['contact'].apply(lambda x: pd.Series(x)),
        venues['location'].apply(lambda x: pd.Series(x)),
    ], axis = 1)],axis=1).set_index(["venue_slug","beer_slug"])

    venue_users = venues_info.groupby(["venue_slug","beer_slug"]).user_name.apply(','.join).rename("user_names")
    venue_rating = venues_info.groupby(["venue_slug","beer_slug"]).rating_score.sum().rename("total_rating")
    venue_counts = venues_info.groupby(["venue_slug","beer_slug"]).size().rename("n_checkins").astype(int)
    venue_last_checkin = venues_info.groupby(["venue_slug","beer_slug"]).created_at.apply(lambda x: x.apply(lambda y:dateparser.parse(y)).max()).rename("last_checkin")


    venues_info_dedup = venues_info.loc[~venues_info.index.duplicated()]
    venues_annotated = venues_info_dedup.join(pd.concat([venue_users, venue_rating, venue_counts,venue_last_checkin],axis=1))

    venues_annotated = venues_annotated.loc[ lambda x: x.venue_url != ""]
    venues_annotated = venues_annotated.loc[ venues_annotated.venue_url.notna()]
    venues_annotated = venues_annotated.loc[ lambda x: x.venue_city != ""]
    venues_annotated = venues_annotated.loc[ venues_annotated.venue_city.notna()]


    cans = pd.read_csv("../_data/cans.csv")
    cans["untappd_id"] = cans.untappd.str.extract('(\d*$)').astype(int)

    venue_beers_annotated = venues_annotated.loc[lambda x: x.venue_id.notna()]
    venue_beers_annotated = venue_beers_annotated.join(cans.set_index("untappd_id")[["code"]], on ="bid")
    venue_beers_annotated = venue_beers_annotated.rename({"code":"can_code"}, axis = "columns")
    venue_beers_annotated["venue_id"] = venue_beers_annotated["venue_id"].astype(int)
    venue_beers_annotated.n_checkins = venue_beers_annotated.n_checkins.astype(int)

    venue_beers_annotated.to_csv("../_data/venue_beers.csv")

    venues_annotated = venue_beers_annotated.reset_index()[["venue_name","venue_slug","primary_category","contact","venue_id","lat","lng","venue_city" , "venue_address","venue_country","venue_city",	"venue_country",	"venue_state",'venue_url']]
    venues_annotated.to_csv("../_data/venues.csv")

# if __name__ == "__main__":
#     os.chdir(os.path.abspath(os.path.dirname(sys.argv[0])))
#     main()

In [19]:
main()


{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 861551896}
{'count': 0, 'items': []}


In [27]:
google_sheet_url_1 = "https://docs.google.com/spreadsheets/d/1cozZyyhcIRuwhdi5Z056_-8LoHC_YR_k-498HTS0-u8/export?format=csv#gid=400302575"
partners_report =pd.read_csv(google_sheet_url,skiprows=0)

In [35]:
google_sheet_url_2 = "https://docs.google.com/spreadsheets/d/1cozZyyhcIRuwhdi5Z056_-8LoHC_YR_k-498HTS0-u8/export?format=csv&id=1cozZyyhcIRuwhdi5Z056_-8LoHC_YR_k-498HTS0-u8&gid=1966703241"
sales_report=pd.read_csv(google_sheet_url_2,skiprows=0)

In [73]:
#accesses can database on airtable
#writes to cans.csv

from airtable import Airtable
base_key = 'appR312nFpbS1UbOH'
table_name = 'Cans'
airtable = Airtable(base_key, table_name, api_key="key78z9GoFTdaPlYd")
record_list = airtable.get_all()
cans_df = pd.DataFrame([record['fields'] for record in record_list]).loc[lambda x: ~x.code.isna()]
cans_df.to_csv("../_data/cans.csv")
for i, r in cans_df.iterrows():
    s = yaml.dump(r.to_dict(), default_flow_style=False)
    with open(f"../_cans/{r.code}.md","w") as f:
        f.write("---\n"),
        f.write(s)
        f.write("\n---\n")

In [72]:
cans_df

,code,beer_cat,untappd,abv,merch_types,beer_style,name,layout,text_description,artist_id,last_brewed,description,series,ad_feature_type,collection_id,ad_feature_id,statement,ad_feature_html,head_graphic
1,p4p,lager,https://untappd.com/b/fab-fermentation-arts-fa...,5,"prints, sold by the artists",pilsner,Pilsner 4 Peace,can-p4p,"American-style Pilsner, with subtle layers of ...","[dye, li, eftekhari, adam, hiya]",1/07,"American-style Pilsner, with subtle layers of ...",p4p,NaN,NaN,NaN,NaN,NaN,NaN
2,trem,ipa,https://untappd.com/b/fab-fermentation-arts-fa...,9,"glassware and Ts, coming soon",new england IPA,Tremendous,can-tremendous,"New England Double IPA (9%), brimming with pas...",[mars],1/17,"New England Double IPA (9%), brimming with pas...",tremendous,NaN,NaN,NaN,NaN,NaN,NaN
3,sqp,ipa,https://untappd.com/b/fab-squid-pro-quo/3568228,8,"museum quality prints, hoodies and stickers",IPA,Squid Pro Quo,can,NaN,[andy],12/4/2019,"<em class=""sqp-text fancy"">SQUID PRO QUO</em> ...",babydictators,prints,156883058772,4405538553940,"""I imagined him floating over a psychedelic su...","<div class=""print-circle-image"" style=""back...",/assets/story/squid_head_only.png
4,bf,ipa,https://untappd.com/b/fab-fermentation-arts-ba...,5.4,"the original artwork, T-shirts and stickers",SESSION IPA,Balloon Factory,can,NaN,[adam],12/3/2019,"<em class=""bf-text fancy"">BALLOON FACTORY</em>...",babydictators,artwork,156883124308,4405832253524,“This baby Trump balloon is a very orange Geor...,"<button type=""button"" class=""btn "" data-toggle...",/assets/story/just_peach.png


In [69]:
# accesses off premises partners from google sheets
# writes to offprem.csv

google_sheet_url_3 = "https://docs.google.com/spreadsheets/d/1cozZyyhcIRuwhdi5Z056_-8LoHC_YR_k-498HTS0-u8/export?format=csv&id=1cozZyyhcIRuwhdi5Z056_-8LoHC_YR_k-498HTS0-u8&gid=45430792"
map_partners = pd.read_csv(google_sheet_url_3, skiprows = 0)
map_partners.to_csv("../_data/offprem_can_locations.csv")